# Assessing Broadband Growth Across the US 💻
## Data Wrangling
### Chester Hitz | Springboard Data Science Career Track | Capstone I

The goal of this section of my capstone project is to capture and arrange the data I will need to determine the factors that lead to the growth of broadband subscription at the county level across the United States. 

-----------------
### The Target Data

The first step in this process is to capture the most important data: rates of broadband subscription by county. A county is a political unit below the state. The data for this is coming from "Form 477 County Data on Internet Access Services", a dataset provided by the Federal Communications Commision (FCC). In this dataset, each county is given a number corresponding to a category for "residential fixed Internet access connections per 1,000 households by county for both service over 200 kbps in at least one direction and service at least 10 Mbps down / 1 Mbps up.". I will refer to this as BSC, or broadband subscription category. The categories are listed as follows, with x representing connections per 1,000 households:
* 0: Zero 
* 1: Zero < x <= 200 
* 2: 200 < x <= 400 
* 3: 400 < x <= 600 
* 4: 600 < x <= 800 
* 5: 800 < x

The data is provided in csv format from the <a href= https://www.fcc.gov/general/form-477-census-tract-data-internet-access-services> FCC website </a>. The years I want to capture are 2011 - 2016, and I want to create a dataframe that captures all the years of all counties as rows with a column for each year showing the broadband category that year. The aim here is to create a dataset that is easily readable for supervised machine learning, with this "broadband category" variable being the target to predict.

I do this below by first importing the 2011 data and using as the basis to merge additional years based on a common key of county IDs. I found the cleanest way to do this on import was to zip up a dictionary with keys set to county_ids and values set to the BSC for that year. Finally, the entire DataFrame is melted around that variable, lengthening the entire dataset by a factor of five.

In [1]:
import csv
import pandas as pd
import random
import numpy as np

# Initial read of 2011 data; removal of unnecessary columns; dropping of counties not in 50 states (US territories)
broadband = pd.read_csv('hs_countydata_dec_2011.csv', header=0, encoding='latin-1', 
                        dtype={'county_fips':str, 'rfc_per_1000_hhs':int})

# county_fips,state,county,statename,countyname,rfc_per_1000_hhs

broadband = broadband[['state','statename','county_fips','rfc_per_1000_hhs']]
broadband = broadband[broadband['state'] < 57] #this drops US territories (Guam etc)

years = ['2011','2012','2013','2014','2015']

for year in years:
    yearFile = 'hs_countydata_dec_' + year + '.csv'
    year_df = pd.read_csv(yearFile, header=0, encoding='latin-1', dtype={'county_fips':str, 'rfc_per_1000_hhs':int})
    year_dict = dict(zip(year_df.county_fips, year_df.rfc_per_1000_hhs))
    broadband[year] = broadband['county_fips'].map(year_dict)
    
broadband.set_index('county_fips')
    
broadband = broadband.melt(id_vars=['state','statename','county_fips'], value_vars = years, var_name='Year', value_name = 'BSC')
print(broadband.head(5))
print(broadband.tail(5))
print(broadband.info())


   state statename county_fips  Year  BSC
0      1   Alabama       01001  2011    4
1      1   Alabama       01003  2011    4
2      1   Alabama       01005  2011    3
3      1   Alabama       01007  2011    2
4      1   Alabama       01009  2011    3
       state statename county_fips  Year  BSC
15710     56   Wyoming       56037  2015    4
15711     56   Wyoming       56039  2015    5
15712     56   Wyoming       56041  2015    5
15713     56   Wyoming       56043  2015    4
15714     56   Wyoming       56045  2015    3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15715 entries, 0 to 15714
Data columns (total 5 columns):
state          15715 non-null int64
statename      15715 non-null object
county_fips    15715 non-null object
Year           15715 non-null object
BSC            15715 non-null int64
dtypes: int64(2), object(3)
memory usage: 613.9+ KB
None


### Factors

#### Colleges

Before moving onto dynamic variables for each year, I would like to add a factor that counts the number of colleges and universities in the county, as I suspect that having a college within the county could be a strong predictor of broadband subscription. I intially tried to do this by running a loop that pinged the Census Geocoder API for the county of each individual college, but that ended up overloading the Census geocoder and was taking hours to process.

Instead, I do this by downloading data with geographical data for all the colleges in the US <a href= https://www.sciencebase.gov/catalog/item/4f4e4acee4b07f02db67fb39> collected here</a>, containing the county ID for each college (the .dbf file was processed externally in QGIS to turn it into a .csv). The unique county IDs are counted with a Counter object, which creates a dictionary containing counts for each county. The dataframe is then  Tract IDs are added to a list, which is passed through a counter object / dictionary, which is then mapped.

In [2]:
from collections import Counter

# import data, select only qualifying schools, create Counter dict
colleges = pd.read_csv('colleges.csv', dtype={'COFIPS':str})
colleges = colleges[colleges['NAICS_DESC']== 'Colleges, Universities, and Professional Schools']
college_dict = Counter(list(colleges.COFIPS))

broadband['colleges'] = broadband['county_fips'].map(college_dict)

#### Rural Percentage

One useful metric provided by the census bureau at the county level is the percentage of the population that can be classified as rural. The exact formulation of this is detailed <a href='https://www2.census.gov/geo/pdfs/reference/ua/Defining_Rural.pdf'>here</a>. For my purposes, I downloaded an Excel file, converted it to csv, and added in a similar fashion to the other factors.

In [3]:
rural_df = pd.read_csv('county_rural.csv', dtype={'2015 GEOID':str})
rural_dict = dict(zip(rural_df['2015 GEOID'], rural_df['2010 Census \rPercent Rural']))
broadband['Rural_Pct'] = broadband['county_fips'].map(rural_dict)

------------
### Additional Factors

Adding additional factors from the census bureau is an interesting challenge. Thousands of data series are available as CSVs and other formats from the census website, but I decided the most dynamic way to add additional data to my dataframe would be through a function that: 1) Submits a request to the Census Bureau API, 2) Recieves the JSON response and parses it, 3) creates a dictionary with the relevant data in the form of: {county_id: value}, 4) maps that dictionary to a new column. This allows for more dynamic addition of factors to the dataframe without having to scrub and clean additional CSVs as they come in.

Two functions to do this are below.

In [4]:
import requests, json
from datetime import datetime

# Defines a basic function to pull census data depending on parameters given
def census_pull(url, parameter):
    payload = {'get': parameter,
                   'for':'county:*',
                   'in': 'state:*',
                   'key': '83a87cce9c801337304138c3da4ee7e290ccb204'}
    resp = requests.get(url, params = payload)
    global data
    data = json.loads(resp.text)

# Main function to pull data and apply it to dataframe.
def census_fill(df, url, parameter, parameter_name, year, dtype=float):
    hold_dict = {}
    # small modification to the url string to access data in 2011
    if year == '2011': mod2011 = '/acs'
    else: mod2011 = ''
    full_url = url.format(str(year), mod2011)
    census_pull(full_url, parameter)
    for item in data[1:]:
        hold_dict[str(item[1])+str(item[2])] = item[0]
    df[parameter_name] = df['county_fips'].map(hold_dict)
    df[parameter_name] = pd.to_numeric(df[parameter_name], errors='coerce')

This next cell collects and organizes the data into the dataframe using the two functions above. The variables here are not guessed a priori, but rather after a few rounds of testing for correlation against the target variable.

In [5]:
metastartTime = datetime.now()
print(metastartTime)

acs_detail = {'Population':'B01001_001E',
              'Income':'B06011_001E',
             'HomePrice':'B25077_001E'}
acs_summary = {'Families': 'DP02_0005E',
               'PctBachelors':'DP02_0064PE',
               'EmploymentRate':'DP03_0026PE',
              'InformationJobs':'DP03_0039PE',
              'AgJobs':'DP03_0033PE',
              'PublicTrans': 'DP03_0021PE',
              'BornInState':'DP02_0089PE',
              'HighSchoolGrads':'DP02_0066PE',
              'MobileHomes':'DP04_0014PE',
              'AptBuildings':'DP04_0013PE',
              'NoEnglish':'DP02_0121PE',
              'InformationJobsPer':'DP03_0039PE',
              'ArmedForcesPer':'DP03_0006PE',
              'Unemployed':'DP03_0005PE',
              'TotalHouseholds':'DP02_0001E',
              'SameHouseResidence':'DP02_0079E'}
CBP = {'Annual Payroll': 'PAYANN',
      'NumberofFirms':'ESTAB'}

new_broadband = pd.DataFrame()
for year in years:
    subset = broadband[broadband['Year'] == year]
    for key in acs_detail:
        census_fill(subset, 'https://api.census.gov/data/{}{}/acs5?', acs_detail.get(key), key, year)
    for key in acs_summary:
        census_fill(subset, 'https://api.census.gov/data/{}{}/acs5/profile?', acs_summary.get(key), key, year)
    for key in CBP:
        census_fill(subset, 'https://api.census.gov/data/{}/cbp?', CBP.get(key), key, year)
    new_broadband = new_broadband.append(subset)
    print(year + ' collection complete.')

broadband = new_broadband
print('End: ' + str(datetime.now() - metastartTime))

2018-04-23 21:20:43.776123


/Users/ChesterHitz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ChesterHitz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2011 collection complete.
2012 collection complete.
2013 collection complete.
2014 collection complete.
2015 collection complete.
End: 0:02:27.415639


#### Population Density

One factor I would like to incorporate is population density (population per sq mile). This data is not available through the API but can be quickly calculated by reading a <a href = https://www.census.gov/geo/maps-data/data/gazetteer2015.html> gazetteer text file </a> and joining it in the dict/map method as the other variables. This is also a good opportunity to create a general purpose divider function for rows.

In [6]:
gazetteer = pd.read_csv('2015_Gaz_counties_national.csv', header=0, dtype={'GEOID':str}, encoding='latin-1')
gazetteer_dict = dict(zip(gazetteer['GEOID'], gazetteer['ALAND_SQMI']))
broadband['TractArea'] = broadband['county_fips'].map(gazetteer_dict)

def divider(df, row1, row2, newrow):
    def subdivider(row):
        if row[row2] != 0 and row[row2] != np.nan and row[row1] != 0 and row[row1] != np.nan:
            return float(row[row1]) / float(row[row2])
        else: return 0
    df[newrow] = df.apply(subdivider, axis=1)
    
divider(broadband, 'Population','TractArea','PopDensity')

#### Data Review



Looking at the data, we can see we do have a number of null values. Further, looking over intermediate .csvs produced I found a number of values of -666666666, which is the census value for null. I replaced those values to be null, then took a look at the NaN values per column. The factor Income has by far the least number of NaN values. I did a quick analysis to see that removing Income would in fact also greatly reduce the number of NaN values overall. 

In [7]:
broadband = broadband.replace(-666666666, np.nan)

In [8]:
print('total null:' , broadband.shape[0] - broadband.dropna().shape[0])
print('percent null:' , 1-(broadband.dropna().shape[0]/broadband.shape[0]))

test_drop = broadband.drop(['Income'], axis=1)

print('total null:' , test_drop.shape[0] - test_drop.dropna().shape[0])
print('percent null:' , 1-(test_drop.dropna().shape[0]/test_drop.shape[0]))

factors = list(broadband.columns)
factors.remove('Income')
broadband = broadband.dropna(subset=(factors))

print('total null:' , broadband.shape[0] - broadband.dropna().shape[0])
print('percent null:' , 1-(broadband.dropna().shape[0]/broadband.shape[0]))

total null: 870
percent null: 0.05536111994909321
total null: 32
percent null: 0.0020362710785872817
total null: 838
percent null: 0.05343365427532998


In [9]:
broadband.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15683 entries, 0 to 15714
Data columns (total 30 columns):
state                 15683 non-null int64
statename             15683 non-null object
county_fips           15683 non-null object
Year                  15683 non-null object
BSC                   15683 non-null int64
colleges              15683 non-null int64
Rural_Pct             15683 non-null float64
Population            15683 non-null float64
Income                14845 non-null float64
HomePrice             15683 non-null float64
Families              15683 non-null float64
PctBachelors          15683 non-null float64
EmploymentRate        15683 non-null float64
InformationJobs       15683 non-null float64
AgJobs                15683 non-null float64
PublicTrans           15683 non-null float64
BornInState           15683 non-null float64
HighSchoolGrads       15683 non-null float64
MobileHomes           15683 non-null float64
AptBuildings          15683 non-null float64


------------
## Conclusion

In this step, I have collected and cleaned a number of factors that I can use to determine the growth of broadband. More importantly, I have created a number of easy to use functions to quickly add and manipulate data as I might need it going forward. It now simply takes one line of code to add additional factors to the dataframe. Next report will dive into the data collected and create visualizations and perform EDA.

In [10]:
print(broadband.shape)
print(broadband.head(10))

(15683, 30)
   state statename county_fips  Year  BSC  colleges  Rural_Pct  Population  \
0      1   Alabama       01001  2011    4         0  42.002162     53944.0   
1      1   Alabama       01003  2011    4         2  42.279099    179523.0   
2      1   Alabama       01005  2011    3         0  67.789635     27546.0   
3      1   Alabama       01007  2011    2         0  68.352607     22746.0   
4      1   Alabama       01009  2011    3         0  89.951502     57140.0   
5      1   Alabama       01011  2011    2         0  51.374382     10877.0   
6      1   Alabama       01013  2011    3         0  71.232157     20860.0   
7      1   Alabama       01015  2011    3         1  33.696826    117614.0   
8      1   Alabama       01017  2011    3         0  49.148034     34375.0   
9      1   Alabama       01019  2011    3         0  85.736273     25819.0   

    Income  HomePrice     ...      NoEnglish  InformationJobsPer  \
0  27807.0   137500.0     ...            0.0                 

In [11]:
broadband.to_csv('broadband.csv')